## Stock Price Prediction

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import math

In [3]:
df = pd.read_csv('stock_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.info()


In [4]:
df.head()

In [5]:
df.describe()

In [6]:
df.isnull().sum()

In [7]:
# Filling missing values with the median of the column
df.fillna(df.select_dtypes(include=[np.number]).median(), inplace=True)

In [8]:
df.isnull().sum()

In [9]:
#Visulaize the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=14)
plt.ylabel('Close Price USD ($)', fontsize=14)
plt.grid()
plt.savefig('close_price_history.png')
plt.show()

In [10]:
#Visualize the trading volume history
plt.figure(figsize=(16,8))
plt.title('Volume History')
plt.plot(df['Volume'])
plt.xlabel('Date', fontsize=14)
plt.ylabel('Volume', fontsize=14)
plt.grid(True)
plt.savefig('trading_volume_history.png')
plt.show()

In [11]:
#Correlation matrix
corr = df.corr()
plt.figure(figsize=(16,8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png')
plt.show()

In [12]:
#Analyze trends and seasonality
from statsmodels.tsa.seasonal import seasonal_decompose

# Fill missing values in 'Close' column
df['Close'] = df['Close'].interpolate(method='linear')

decompose = seasonal_decompose(df['Close'], model='multiplicative', period=20)
fig = decompose.plot()
fig.set_size_inches(16,8)
plt.savefig('seasonal_decompose.png')
plt.show()

In [13]:
#Check for stationarity
from statsmodels.tsa.stattools import adfuller
result = adfuller(df['Close'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print('Critical Values:')
for key, value in result[4].items():
    print(f'{key}:{value}')

### Feature Engineering

In [34]:
# Calculate moving averages
df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA20'] = df['Close'].rolling(window=20).mean()

# Moving average crossovers (buy/sell signals)
df['MA5_20_Cross'] = (df['MA5'] > df['MA20']).astype(int)
df['MA20_50_Cross'] = (df['MA20'] > df['MA50']).astype(int)

# Distance from moving averages (normalized)
df['Dist_MA20'] = (df['Close'] - df['MA20']) / df['MA20']
df['Dist_MA50'] = (df['Close'] - df['MA50']) / df['MA50']

# Calculate price momentum
df['Momentum'] = df['Close'] - df['Close'].shift(5)
df['Momentum_10'] = df['Close'] - df['Close'].shift(10)
df['Momentum_20'] = df['Close'] - df['Close'].shift(20)

# Calculate volatility (standard deviation over a window)
df['Volatility'] = df['Close'].rolling(window=20).std()

# Price rate of change at different timeframes
df['ROC_5'] = df['Close'].pct_change(periods=5) * 100
df['ROC_10'] = df['Close'].pct_change(periods=10) * 100
df['ROC_20'] = df['Close'].pct_change(periods=20) * 100

In [35]:
# Drop NaN values created by the indicators
df.dropna(inplace=True)

In [36]:
# Visualize the new features
plt.figure(figsize=(16, 8))
plt.title('Technical Indicators')
plt.plot(df['Close'], label='Close')
plt.plot(df['MA20'], label='MA20')
plt.plot(df['MA50'], label='MA50')
plt.legend()
plt.savefig('technical_indicators.png')
plt.show()

In [37]:
# Visualize MACD
plt.figure(figsize=(16, 8))
plt.title('MACD')
plt.plot(df['MACD'], label='MACD')
plt.plot(df['MACD_Signal'], label='Signal Line')
plt.legend()
plt.savefig('macd.png')
plt.show()

### Data prepaeration for LSTM -

In [122]:
# Define the target variable: 5-day future closing price
df['Target'] = df['Close'].shift(-5)  # 5 days into the future
df.dropna(inplace=True)  # Drop rows with NaN targets

# Select features
features = [ 'Close', 'High','Low','Volume','MA5', 'MA20', 'Momentum', 'Volatility', 'RSI', 'MACD',
            'MACD_Signal', 'MA5_20_Cross', 'MA20_50_Cross', 'Dist_MA20', 'Dist_MA50',
            'ROC_5', 'ROC_10', 'ROC_20']
X = df[features].values
y = df['Target'].values

In [123]:
# Split data into training and testing sets (keeping time order)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [124]:
# Normalize the data 
X_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))

In [125]:
# Create sequences for LSTM input (lookback window)
def create_sequences(X, y, time_steps=20):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 20  # Number of previous time steps to use as input
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, time_steps)

In [126]:
# Check the shapes
print(f"X_train_seq shape: {X_train_seq.shape}")  # (samples, time steps, features)
print(f"y_train_seq shape: {y_train_seq.shape}")  # (samples, 1)

Build the model

In [127]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))  # Predicting a single value (close price)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [128]:
# Print input shapes to debug
print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")

# Rebuild the model with proper input shape
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Train the model with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [129]:
# Plot training history
plt.figure(figsize=(16, 8))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('model_training_history.png')
plt.show()

In [120]:
# Make predictions
y_pred_scaled = model.predict(X_test_seq)

# Check shapes
print(f"y_pred_scaled shape: {y_pred_scaled.shape}")
print(f"y_test_seq shape: {y_test_seq.shape}")

# If the model is outputting multiple values per sample, take only the first one
if len(y_pred_scaled.shape) > 2 or (len(y_pred_scaled.shape) == 2 and y_pred_scaled.shape[1] > 1):
	y_pred_scaled = y_pred_scaled[:, 0].reshape(-1, 1)

# Inverse transform to get actual price values
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_test_actual = y_scaler.inverse_transform(y_test_seq)

# Calculate error metrics
rmse = math.sqrt(mean_squared_error(y_test_actual, y_pred))
mae = mean_absolute_error(y_test_actual, y_pred)
mape = np.mean(np.abs((y_test_actual - y_pred) / y_test_actual)) * 100

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Absolute Percentage Error: {mape}%')

In [130]:
# Calculate directional accuracy (whether the price direction was predicted correctly)
y_test_direction = np.diff(y_test_actual.flatten())
y_pred_direction = np.diff(y_pred.flatten())
directional_accuracy = np.mean((y_test_direction > 0) == (y_pred_direction > 0)) * 100
print(f'Directional Accuracy: {directional_accuracy}%')

### Using N-Beats

In [131]:
# 1. Data Preparation and Normalization
# -------------------------------------
# Select the target column (e.g., 'Close' price for stocks)
target_column = 'Close'
data = df[[target_column]].copy()

# Apply normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [132]:
# 2. Create Sequences for Time Series Modeling
# -------------------------------------------
def create_sequences(data, lookback, forecast_horizon):
    """
    Create input/output sequences for time series forecasting
    
    Parameters:
    data: normalized data array
    lookback: number of past time steps to use as input
    forecast_horizon: number of future time steps to predict
    
    Returns:
    X: input sequences of shape (samples, lookback)
    y: output sequences of shape (samples, forecast_horizon)
    """
    X, y = [], []
    for i in range(len(data) - lookback - forecast_horizon + 1):
        X.append(data[i:(i + lookback)].flatten())
        y.append(data[(i + lookback):(i + lookback + forecast_horizon)].flatten())
    return np.array(X), np.array(y)

# Define sequence parameters
lookback = 30       # Use 30 days of historical data for prediction
forecast_horizon = 5 # Predict 5 days into the future

# Create sequences
X, y = create_sequences(scaled_data, lookback, forecast_horizon)
print(f"Input sequence shape: {X.shape}")
print(f"Output sequence shape: {y.shape}")

In [133]:
# 3. Train-Test Split
# ------------------
# Use the last 20% of data for testing to preserve temporal order
split_idx = int(len(X) * 0.8)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")

In [134]:
# 4. Create N-BEATS Block Building Function
# ----------------------------------------
def nbeats_block(x, units, theta_dim, share_weights=False, block_name=None):
    """
    Create an N-BEATS block
    
    Parameters:
    x: input tensor
    units: number of hidden units
    theta_dim: dimension of theta layer
    share_weights: whether to share weights
    block_name: name of the block
    
    Returns:
    backcast: backcast output tensor
    forecast: forecast output tensor
    """
    # Block contains 4 fully connected layers
    for _ in range(4):
        x = Dense(units, activation='relu')(x)
    
    # Output is split into backcast and forecast
    theta_b = Dense(theta_dim, name=f'{block_name}_theta_b')(x)
    theta_f = Dense(theta_dim, name=f'{block_name}_theta_f')(x)
    
    # Reshape to match input shape for backcast and forecast_horizon for forecast
    backcast = Dense(lookback, name=f'{block_name}_backcast')(theta_b)
    forecast = Dense(forecast_horizon, name=f'{block_name}_forecast')(theta_f)
    
    return backcast, forecast

In [136]:
# 5. Build the N-BEATS Model
# -------------------------
from tensorflow.keras.layers import Input, Dense, Subtract, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_nbeats_model(lookback, forecast_horizon, stacks=2, blocks_per_stack=3, units=256, share_weights=False):
    """
    Build the N-BEATS model
    
    Parameters:
    lookback: input sequence length
    forecast_horizon: output sequence length
    stacks: number of stacks
    blocks_per_stack: number of blocks per stack
    units: number of hidden units
    share_weights: whether to share weights between blocks
    
    Returns:
    model: compiled N-BEATS model
    """
    input_layer = Input(shape=(lookback,), name='input_layer')
    
    # Initial backcast is the input
    backcast = input_layer
    forecast = None
    
    # For collecting forecasts from each block
    forecasts = []
    
    # Stack loop
    for stack_id in range(stacks):
        # Block loop
        for block_id in range(blocks_per_stack):
            # Apply N-BEATS block
            backcast_block, forecast_block = nbeats_block(
                backcast, 
                units=units, 
                theta_dim=lookback,
                share_weights=share_weights,
                block_name=f'stack_{stack_id}_block_{block_id}'
            )
            
            # Collect forecasts
            forecasts.append(forecast_block)
            
            # Subtract the backcast from the input to get the residual
            backcast = Subtract(name=f'subtract_{stack_id}_{block_id}')([backcast, backcast_block])
    
    # Double check stack and blocks_per_stack
    assert len(forecasts) == stacks * blocks_per_stack
    
    # Sum all the forecasts
    if len(forecasts) > 1:
        forecast = forecasts[0]
        for i in range(1, len(forecasts)):
            forecast = Concatenate(axis=1, name=f'forecast_add_{i}')([forecast, forecasts[i]])
    else:
        forecast = forecasts[0]
    
    # Build model
    model = Model(inputs=input_layer, outputs=forecast)
    model.compile(
        loss='mse',
        optimizer=Adam(learning_rate=0.001),
        metrics=['mae']
    )
    
    return model

# Build the model
model = build_nbeats_model(
    lookback=lookback,
    forecast_horizon=forecast_horizon,
    stacks=2,
    blocks_per_stack=3,
    units=128
)

# Print model summary
model.summary()

In [141]:
# 6. Model Training
# ---------------
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Add, Lambda
import tensorflow as tf

# Check shapes
print(f"X shape: {X_train.shape}")
print(f"y shape: {y_train.shape}")

# For the N-BEATS model, we need to ensure the target has the correct shape
# Take only the first forecast step for now (we can use more as needed)
y_train_reshaped = y_train[:, 0].reshape(-1, 1)
print(f"Reshaped y shape: {y_train_reshaped.shape}")

# Rebuild the model with clear TF graph to avoid variable creation issues
tf.keras.backend.clear_session()

# Recreate the model using the Functional API more explicitly
input_layer = Input(shape=(lookback,), name='input_layer')
forecasts = []

# Initial backcast is the input
backcast = input_layer

# Build stacks and blocks
for stack_id in range(2):  # 2 stacks
    for block_id in range(3):  # 3 blocks per stack
        # First dense layers
        x = Dense(128, activation='relu')(backcast)
        x = Dense(128, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        x = Dense(128, activation='relu')(x)
        
        # Output layers
        theta_b = Dense(lookback, name=f'stack_{stack_id}_block_{block_id}_theta_b')(x)
        theta_f = Dense(lookback, name=f'stack_{stack_id}_block_{block_id}_theta_f')(x)
        
        backcast_block = Dense(lookback, name=f'stack_{stack_id}_block_{block_id}_backcast')(theta_b)
        forecast_block = Dense(forecast_horizon, name=f'stack_{stack_id}_block_{block_id}_forecast')(theta_f)
        
        # Collect forecast
        forecasts.append(forecast_block)
        
        # Residual connection
        backcast = Subtract(name=f'subtract_{stack_id}_{block_id}')([backcast, backcast_block])

# Average the forecasts from all blocks
if len(forecasts) > 1:
    final_forecast = forecasts[0]
    for i in range(1, len(forecasts)):
        final_forecast = Add(name=f'forecast_add_{i}')([final_forecast, forecasts[i]])
    final_forecast = Lambda(lambda x: x / len(forecasts))(final_forecast)
else:
    final_forecast = forecasts[0]

# Create and compile model
model = Model(inputs=input_layer, outputs=final_forecast)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

model.summary()

# Set up callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train_reshaped,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [143]:
# 8. Model Evaluation
# -----------------
# Import r2_score
from sklearn.metrics import r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Inverse transform predictions and actual values to original scale
y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(y_test.shape)
y_pred_orig = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(y_pred.shape)

# Calculate error metrics on original scale data
mse = mean_squared_error(y_test_orig, y_pred_orig)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig.reshape(-1), y_pred_orig.reshape(-1))

print("\nModel Evaluation Metrics (Original Scale):")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")

# Calculate MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_test_orig - y_pred_orig) / y_test_orig)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

In [ ]:
# # Calculate and Visualize Directional Accuracy
# # -------------------------------------------

# # First, make predictions if not already done
# y_pred = model.predict(X_test)

# # Reshape if needed for directional accuracy calculation
# y_test_reshaped = y_test if y_test.ndim == 2 else y_test.reshape(-1, forecast_horizon)
# y_pred_reshaped = y_pred if y_pred.ndim == 2 else y_pred.reshape(-1, forecast_horizon)

# # Inverse transform to original scale if required
# y_test_orig = scaler.inverse_transform(y_test_reshaped.reshape(-1, 1)).reshape(y_test_reshaped.shape)
# y_pred_orig = scaler.inverse_transform(y_pred_reshaped.reshape(-1, 1)).reshape(y_pred_reshaped.shape)

# # Calculate directional accuracy for each forecast horizon
# directional_accuracy = np.zeros(forecast_horizon)

# for horizon in range(forecast_horizon):
#     # Get the actual price movements (up or down)
#     # We compare consecutive predictions for the same horizon 
#     actual_direction = np.sign(np.diff(y_test_orig[1:, horizon] - y_test_orig[:-1, horizon]))
#     predicted_direction = np.sign(np.diff(y_pred_orig[1:, horizon] - y_pred_orig[:-1, horizon]))
    
#     # Calculate accuracy (ignoring zero changes)
#     non_zero_indices = actual_direction != 0
#     if np.sum(non_zero_indices) > 0:
#         correct_directions = np.sum((actual_direction[non_zero_indices] == predicted_direction[non_zero_indices]))
#         directional_accuracy[horizon] = correct_directions / np.sum(non_zero_indices)

# # Print overall directional accuracy
# overall_da = np.mean(directional_accuracy)
# print(f"\nDirectional Accuracy Analysis:")
# print(f"Overall Directional Accuracy: {overall_da:.4f} ({overall_da*100:.2f}%)")

# for h in range(forecast_horizon):
#     print(f"Horizon {h+1} Directional Accuracy: {directional_accuracy[h]:.4f} ({directional_accuracy[h]*100:.2f}%)")

### Final Visualization

In [99]:
# Visualize predictions vs actual values
plt.figure(figsize=(14, 7))
plt.title('Stock Price Prediction')
plt.plot(y_test_actual, label='Actual Prices')
plt.plot(y_pred, label='Predicted Prices')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.savefig('prediction_vs_actual.png')
plt.show()

In [100]:
# Calculate trading performance
initial_capital = 10000
position = 0
portfolio_value = [initial_capital]
returns = []

In [101]:
# Simple trading strategy: Buy when predicted price increases, sell when predicted price decreases
for i in range(1, len(y_pred)):
    # Predicted price change
    price_change = y_pred[i] - y_pred[i-1]
    
    if price_change > 0 and position == 0:  # Buy signal
        position = portfolio_value[-1] / y_test_actual[i]
        portfolio_value.append(portfolio_value[-1])  # No change until we sell
    elif price_change < 0 and position > 0:  # Sell signal
        portfolio_value.append(position * y_test_actual[i])
        position = 0
    else:
        if position > 0:  # If holding position, update portfolio value
            portfolio_value.append(position * y_test_actual[i])
        else:
            portfolio_value.append(portfolio_value[-1])
    
    returns.append((portfolio_value[-1] - portfolio_value[-2]) / portfolio_value[-2])

In [102]:
# Calculate trading metrics
cumulative_return = (portfolio_value[-1] - initial_capital) / initial_capital * 100
annualized_return = (1 + (portfolio_value[-1] - initial_capital) / initial_capital) ** (252 / len(y_pred)) - 1
sharpe_ratio = np.mean([r.item() if isinstance(r, np.ndarray) else r for r in returns]) / np.std([r.item() if isinstance(r, np.ndarray) else r for r in returns]) * np.sqrt(252)
max_drawdown = np.max(np.maximum.accumulate([pv.item() if isinstance(pv, np.ndarray) else pv for pv in portfolio_value]) - [pv.item() if isinstance(pv, np.ndarray) else pv for pv in portfolio_value]) / np.max([pv.item() if isinstance(pv, np.ndarray) else pv for pv in portfolio_value])

# Extract scalar values from numpy arrays
cumulative_return = cumulative_return.item() if isinstance(cumulative_return, np.ndarray) else cumulative_return
annualized_return = annualized_return.item() if isinstance(annualized_return, np.ndarray) else annualized_return
sharpe_ratio = sharpe_ratio.item() if isinstance(sharpe_ratio, np.ndarray) else sharpe_ratio
max_drawdown = max_drawdown.item() if isinstance(max_drawdown, np.ndarray) else max_drawdown

print(f'Cumulative Return: {cumulative_return:.2f}%')
print(f'Annualized Return: {annualized_return:.2f}%')
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')
print(f'Maximum Drawdown: {max_drawdown:.2f}')

In [146]:
# Convert any numpy arrays in portfolio_value to scalar values
portfolio_value = [pv.item() if isinstance(pv, np.ndarray) else pv for pv in portfolio_value]

# Plot portfolio value over time
plt.figure(figsize=(16, 8))
plt.title('Portfolio Value')
plt.plot(portfolio_value)
plt.xlabel('Trading Days')
plt.ylabel('Portfolio Value ($)')
plt.grid(True)
plt.savefig('portfolio_performance.png')
plt.show()

### Feature Importance Analysis

In [50]:
# For LSTM, direct feature importance is not straightforward
# One approach is to use permutation importance
from sklearn.inspection import permutation_importance

# We need to adapt this for LSTM's sequential data
# Create a function to make predictions and calculate RMSE
def predict_and_score(model, X, y):
    y_pred = model.predict(X)
    return -mean_squared_error(y, y_pred)  # Negative because we want higher values to be more important

# Baseline score
baseline_score = predict_and_score(model, X_test_seq, y_test_seq)

# Initialize importance scores
importance_scores = {}

In [51]:
# For each feature, permute its values and measure the impact
for i in range(X_train_seq.shape[2]):
    # Make a copy of the test data
    X_permuted = X_test_seq.copy()
    
    # Shuffle the values of the current feature across all sequences
    for j in range(X_test_seq.shape[0]):
        np.random.shuffle(X_permuted[j, :, i])
    
    # Calculate new score with permuted feature
    new_score = predict_and_score(model, X_permuted, y_test_seq)
    
    # The importance is the difference in scores
    importance = baseline_score - new_score
    importance_scores[features[i]] = importance

In [52]:
# Normalize importances
max_importance = max(importance_scores.values())
for feature in importance_scores:
    importance_scores[feature] /= max_importance

# Plot feature importances
plt.figure(figsize=(12, 8))
plt.barh(list(importance_scores.keys()), list(importance_scores.values()))
plt.xlabel('Normalized Importance')
plt.title('Feature Importance')
plt.savefig('feature_importance.png')
plt.show()